<a href="https://colab.research.google.com/github/M-H-Amini/GAN-Webinars/blob/main/DCGAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  In The Name Of ALLAH
#  Generative Adversarial Networks
#  PythonChallenge.ir
#  Mohammad Hossein Amini (mhamini@aut.ac.ir)
#  Lecture 1 - DCGAN

In this lecture we'll be implementing a DCGAN. The theoretical stuff has been discussed in the video. Let's see how things work in practice!

In [ ]:
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, Activation, LeakyReLU
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

#  Dataset Preparation
We would use the famous **MNIST** dataset for handwritten digits. Let's assume we just want to generate a specific digit.

In [ ]:
(Xtrain, ytrain), (_, _) = mnist.load_data()
Xtrain = (np.expand_dims(Xtrain[ytrain==8], 3) - 127.5) / 127.5

#  Visualization
In order to visualize results, we would implement ```show``` function.

In [ ]:
def show(X, r=4, c=4):
  fig, ax = plt.subplots(r, c, True, True)
  for i in range(r):
    for j in range(c):
      ax[i][j].imshow(X[i*c + j, :, :, 0])
  plt.show()

show(Xtrain, 2, 3)

We determine image shapes and the our noise dimension.

In [ ]:
z_dim = 100
img_shape = 28, 28

#  Discriminator
Let's build **discriminator** now. We use an MLP as the discriminator.

In [ ]:
def buildDisc(img_shape=(28, 28)):
  model = Sequential()
  model.add(Conv2D(64, 3, 2, 'same', input_shape=(*img_shape, 1)))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Conv2D(128, 3, 2, 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Flatten())
  model.add(Dense(1))
  return model

# Generator
Again, we use an MLP for the generator.

In [ ]:
def buildGen(z_dim=100, img_shape=(28, 28)):
  model = Sequential()
  model.add(Dense(7 * 7 * 128, input_shape=(z_dim,)))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Reshape((7, 7, 128)))
  model.add(Conv2DTranspose(64, (5, 5), 2, padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Conv2DTranspose(1, (5, 5), 2, padding='same', activation='tanh'))
  return model

#  GAN
Time to build a model for **GAN**. We build it by cascading the generator and the discriminator.

In [ ]:
def buildGan(disc, gen):
  model = Sequential()
  model.add(gen)
  model.add(disc)
  return model

In [ ]:
disc = buildDisc()
disc.compile(optimizer=Adam(1e-4), loss=BinaryCrossentropy(True), metrics=['acc'])

gen = buildGen()

disc.trainable = False
gan = buildGan(disc, gen)
gan.compile(optimizer=Adam(1e-4), loss=BinaryCrossentropy(True))

#  Training

In [ ]:
batch_size = 128
for i in range(20000):
  indexes = np.random.permutation(len(Xtrain))[:batch_size]
  Xreal = Xtrain[indexes]
  yreal = np.ones((batch_size,))
  noise = np.random.randn(batch_size, z_dim)
  Xfake = gen.predict(noise)
  yfake = np.zeros_like(yreal)
  Xdisc = np.concatenate((Xreal, Xfake))
  ydisc = np.concatenate((yreal, yfake))

  d_loss, d_acc = disc.train_on_batch(Xdisc, ydisc)


  noise = np.random.randn(batch_size, z_dim)
  g_loss = gan.train_on_batch(noise, yreal)

  if not((i+1)%50):
    print(f'Iteration {i+1}:\tD acc: {d_acc}\tD loss: {d_loss}\tG loss: {g_loss}')
    noise = np.random.randn(batch_size, z_dim)
    Xfake = gen.predict(noise)
    show(Xfake)
